In [28]:
import pandas as pd

data = pd.read_json("https://data.nsw.gov.au/data/api/3/action/datastore_search?resource_id=2776dbb8-f807-4fb2-b1ed-184a6fc2c8aa&limit=6000")
data = pd.json_normalize(data.result.records)
data = data[["notification_date", "lga_code19"]]
data.columns = ["date", "code"]
data.date = pd.to_datetime(data.date)
data["cases"] = [1] * len(data)
data.describe(datetime_is_numeric=True)
data = data.groupby(["date", "code"]).sum().reset_index()
data.code = data.code.astype("int")
data.dropna(inplace=True)
data.sort_values("date", inplace=True)

act_data = pd.read_csv("./actCases.csv")
act_data.date = pd.to_datetime(act_data.date, format="%d/%m/%Y")
act_data["code"] = 89399

data = pd.concat([data, act_data], ignore_index=True)
date_start = data.date.min()
date_end = data.date.max()

data_chart = pd.DataFrame(columns=["date", "code", "cases", "recent"])

count = 0
while date_start + pd.Timedelta(count, unit="d") <= date_end:
    data_temp = data[data.date == date_start + pd.Timedelta(count, unit="d")]
    count = count + 1

# for i in data.index:    
#     data_temp = data[(data.code == data.at[i, "code"]) & (data.date > data.at[i, "date"] - pd.Timedelta("14 days")) & (data.date <= data.at[i, "date"])]
#     data.at[i, "recent"] = data_temp.cases.sum()

# data
# data.to_csv("./cases.csv", index=False)

11300
14500
16260
16550
12850
14000
10500
11570
16700
17150
17420
15990
17200
13310
12380
10900
14900
18500
18250
14170
14700
11520
10750
16350
16150
11650
14100
18050
13800
11500
11450
18350
15950
11800
11720
12930
11350
18450
15350
14650
15240
16950
15900
10470
14400
17550
15050
12700
17100
14920
11730
16380
16490
16900
10550
12750
16400
17310
17400
10050
10250
10850
15270
15700
18020
14870
12390
14850
14200
14350
17750
17000
11400
12160
13450
12870
15850
17620
18100
18400
17040
13340
13010
15650
17950
18710
15560
17640
10130
10600
13850
10650
15800
16610
17650
11250
13910
13550
16200
15300
17080
14300
17350
12150
89399
